In [9]:
import numpy as np
import math
from tracklet import Tracklet
from tracklet import TrackletCollection

In [6]:
gt_box = np.load('./one_frame_data/gt_boxes3d.npy')
print gt_box
print gt_box.shape

[[[  1.13325310e+01   4.20234966e+00  -1.74235904e+00]
  [  1.22772589e+01   2.64294267e+00  -1.74235904e+00]
  [  1.60695038e+01   4.94038010e+00  -1.74235904e+00]
  [  1.51247759e+01   6.49978685e+00  -1.74235904e+00]
  [  1.13325310e+01   4.20234966e+00   2.57640958e-01]
  [  1.22772589e+01   2.64294267e+00   2.57640958e-01]
  [  1.60695038e+01   4.94038010e+00   2.57640958e-01]
  [  1.51247759e+01   6.49978685e+00   2.57640958e-01]]

 [[  5.13593483e+00  -1.30706954e+00  -1.70471942e+00]
  [  5.22208071e+00  -2.12714815e+00  -1.70471942e+00]
  [  6.99755239e+00  -1.94064200e+00  -1.70471942e+00]
  [  6.91140652e+00  -1.12056351e+00  -1.70471942e+00]
  [  5.13593483e+00  -1.30706954e+00   3.43430750e-02]
  [  5.22208071e+00  -2.12714815e+00   3.43430750e-02]
  [  6.99755239e+00  -1.94064200e+00   3.43430750e-02]
  [  6.91140652e+00  -1.12056351e+00   3.43430750e-02]]

 [[  8.16266346e+00  -6.07901144e+00  -1.70272017e+00]
  [  8.41108894e+00  -6.80559635e+00  -1.70272017e+00]
  [  9

In [20]:
def box_to_tracklet(box, frame_number):
    lv2d = box[1][:2] - box[0][:2] # x,y component of l vector
    l = np.linalg.norm(lv2d)
    w = np.linalg.norm(box[3][:2] - box[0][:2])
    h = box[4][2] - box[0][2]
    center = (box[0] + box[6]) * 0.5
    lv2dn = lv2d / l # normalize
    yaw = 0
    if lv2dn[0] < 0.0001:
        yaw = math.pi if lv2dn[1] > 0 else -math.pi
    else:
        yaw = math.atan2(lv2dn[1], lv2dn[0])    
    t = Tracklet('Car', l, w, h)
    t.first_frame = frame_number
    p = {'tx': center[0], 'ty': center[1], 'tz': center[2], 'rx': 0, 'ry': 0, 'rz': yaw}
    t.poses.append(p)
    return t

In [21]:
def write_tracklet(filename, box_list):
    tracklet_col = TrackletCollection()
    for i in range(len(box_list)):
        t = box_to_tracklet(box_list[i], i)
        tracklet_col.tracklets.append(t)
        tracklet_col.write_xml(filename)

In [22]:
write_tracklet('./one_frame_data/tracklet.xml', gt_box)